In [13]:
require 'rest-client' 

def fetch(url, headers = {accept: "*/*"}, user = "", pass="")
  response = RestClient::Request.execute({
    method: :get,
    url: url.to_s,
    user: user,
    password: pass,
    headers: headers})
  return response
  
  rescue RestClient::ExceptionWithResponse => e
    $stderr.puts e.inspect
    response = false
    return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
  rescue RestClient::Exception => e
    $stderr.puts e.inspect
    response = false
    return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
  rescue Exception => e
    $stderr.puts e.inspect
    response = false
    return response  # now we are returning 'False', and we will check that with an \"if\" statement in our main code
end 



def get_AGI_Locus(path)
  locus_code= []
  locus = File.open(path, mode: 'r')
  locus.readlines[1..30].each do |line|
    code = line.strip.split("\n")
  locus_code |= code
  end 
  return locus_code
end  

def get_interacction(all_locus)
  interacctions = Hash.new
  #all_int_gen = Hash.new

    all_locus.each do |locus|
    res = fetch("http://bar.utoronto.ca:9090/psicquic/webservices/current/search/interactor/#{locus}/?firstResult=0&maxResults=30&format=tab25");  #restritions: 9090-> species, maxresult -> 30
    body = res.body.split("\n") #each interaction is separated by \n
      #puts locus
    all_interacctions = []
    all_int_gen = Hash.new
    body.each do |elem|
      elem = elem.split("\t")
      elem[-1].to_s =~ /(\d.\d+)/
      score = $1
      if score.to_f > 0.5  #significative medium-high interaction above 0.4
        elem[2] =~ /(A[Tt]\d[Gg]\d\d\d\d\d)/
        gen1 = $1
        elem[3] =~ /(A[Tt]\d[Gg]\d\d\d\d\d)/
        gen2 = $1
        next if gen1.nil?||gen2.nil?
        #puts gen1,gen2

        if gen1.upcase != locus.upcase #I use upcase because all locus have a T instead of the t in the gen interactors
          all_interacctions.push(gen1)
        else
          all_interacctions.push(gen2)
        end 
      end
    end
    next if all_interacctions[0].nil?
    all_int_gen[locus] = all_interacctions
    puts "primeras interacciones"
    puts all_int_gen
    puts "--------------"
    
    include_int = []
    new_int = Hash.new
  
   all_int_gen.each do |key1, locus2|
    locus2.each do |locus_2|
    res = fetch("http://bar.utoronto.ca:9090/psicquic/webservices/current/search/interactor/#{locus_2}/?firstResult=0&maxResults=30&format=tab25");  #restritions: 9090-> species, maxresult -> 30
    body = res.body.split("\n") #each interaction is separated by \n
    interac_2 = []
    all_int_2 = Hash.new
    body.each do |elem|
      elem = elem.split("\t")
      elem[-1].to_s =~ /(\d.\d+)/
      score = $1
      if score.to_f > 0.5  #significative medium-high interaction above 0.4
        elem[2] =~ /(A[Tt]\d[Gg]\d\d\d\d\d)/
        gen1 = $1
        elem[3] =~ /(A[Tt]\d[Gg]\d\d\d\d\d)/
        gen2 = $1
        next if gen1.nil?||gen2.nil?
        #puts gen1,gen2

        if gen1.upcase != locus_2.upcase && gen1.upcase != locus.upcase #I use upcase because all locus have a T instead of the t in the gen interactors
          interac_2.push(gen1)
        else
          if gen2.upcase != locus.upcase
          interac_2.push(gen2)
          end
        end 
      end
    end
    next if interac_2[0].nil?
    all_int_2[locus_2] = interac_2
     #print("second interacctions\n")
    #puts all_int_2
     
    all_int_2.each do |key, values|
      all_locus = all_locus.map{|x| x.upcase}
        if all_locus.include?(key.upcase)
          if key.upcase != locus.upcase  
          print("included another gen on the list\n")
          interacctions[key1] = [key]
          end
        end
    
      values.each do |value|
        if all_locus.include?(value.upcase)
          if value.upcase != key.upcase 
          print("included\n")
          next if include_int.include?(value)
          include_int.push(value)
            puts include_int
            new_int[key] = include_int
            #puts new_int
          end
          puts new_int
      end
    
    end
            next if new_int[key].nil?
            interacctions[key1]= new_int
            puts "\ngetting interacctions"
            puts interacctions
            puts "---------"
        end

  end

end 
  end
return interacctions
end 


def get_Network(interactions)
  interaction_keys = interactions.keys.map{|x| x.upcase}
  interactions.each do |key, values|
    values.each do |key2, value2|
      if interactions_keys.include?(value2.upcase)
      bigger_int = interactions_keys.select{|x| x == value2.upcase} 
        interactions[key][key2].push(interactions[bigger_int])
      end 
    end
  end
  return interactions
end
        
        

:get_Network

In [12]:
get_interacction(get_AGI_Locus('./ArabidopsisSubNetwork_GeneList.txt'))

primeras interacciones
{"AT5g54270"=>["At4g37140", "At5g12980", "At4g35350", "At1g22300", "At5g45870", "At1g10430", "At3g32920", "At4g37930", "At1g79870", "At3g44680", "At4g24220", "At4g34840", "At5g26780", "At1g69960", "At2g20160"]}
--------------
primeras interacciones
{"AT2g13360"=>["At2g13360", "At3g14415"]}
--------------
primeras interacciones
{"AT4g05180"=>["At4g25200"]}
--------------
primeras interacciones
{"AT2g45170"=>["At3g45240", "At1g66240", "At4g17190", "At5g49970", "At3g18850", "At3g22290", "At4g33710", "At4g39200", "At2g37420", "At4g22120", "At5g24090", "At3g54840", "At3g09630", "At3g47290", "At5g14060", "At1g07180", "At5g09660", "At3g53960", "At1g74710", "At4g14350", "At5g65980", "At1g25155", "At5g15450", "At5g16980", "At1g15440", "At5g13860", "At2g36170", "At2g33210", "At4g37930"]}
--------------
included
["At1g23740"]
{"At1g07180"=>["At1g23740"]}

getting interacctions
{"AT2g45170"=>{"At1g07180"=>["At1g23740"]}}
---------
primeras interacciones
{"AT1g80440"=>["At3g1

{"AT2g45170"=>{"At1g07180"=>["At1g23740"]}, "AT5g04140"=>{"At4g34230"=>["At4g17090"]}, "AT4g17090"=>{"At4g34230"=>["At5g04140"]}, "AT1g23740"=>{"At4g33710"=>["At2g45170"]}}

In [ ]:
get_Network(get_interacction(get_AGI_Locus('./ArabidopsisSubNetwork_GeneList.txt')))

primeras interacciones
{"AT5g54270"=>["At4g37140", "At5g12980", "At4g35350", "At1g22300", "At5g45870", "At1g10430", "At3g32920", "At4g37930", "At1g79870", "At3g44680", "At4g24220", "At4g34840", "At5g26780", "At1g69960", "At2g20160"]}
--------------
primeras interacciones
{"AT2g13360"=>["At2g13360", "At3g14415"]}
--------------
primeras interacciones
{"AT4g05180"=>["At4g25200"]}
--------------
primeras interacciones
{"AT2g45170"=>["At3g45240", "At1g66240", "At4g17190", "At5g49970", "At3g18850", "At3g22290", "At4g33710", "At4g39200", "At2g37420", "At4g22120", "At5g24090", "At3g54840", "At3g09630", "At3g47290", "At5g14060", "At1g07180", "At5g09660", "At3g53960", "At1g74710", "At4g14350", "At5g65980", "At1g25155", "At5g15450", "At5g16980", "At1g15440", "At5g13860", "At2g36170", "At2g33210", "At4g37930"]}
--------------
included
["At1g23740"]
{"At1g07180"=>["At1g23740"]}

getting interacctions
{"AT2g45170"=>{"At1g07180"=>["At1g23740"]}}
---------
primeras interacciones
{"AT1g80440"=>["At3g1

NameError: undefined local variable or method `interactions_keys' for main:Object
Did you mean?  interaction_keys
               interactions